# Milestone 3

## Project Statement


While soccer is one of the most popular sports in the world, its analytics is lackluster in comparison to other popular sports such as baseball, American football, hockey, and basketball. This is due to the fact that aside from the World Cup, international teams don’t play each other very frequently. Therefore, there is a lack of high quality data that can be immediately fed into models to perform predictions.

The World Cup is composed of 64 matches in total - 48 matches in group stages and 16 matches in knockout (15 + 1 for third place). We plan to predict the outcome of each of the 64 matches independently instead of predicting which teams proceed in each round. This strategy allows our results to be comparable across models. By framing the problem in this way, we plan to approach this problem as a classification problem. Each game can be treated as a multi-class classification problem, where there are three outcomes: win for the home team (or team 1, indicated with 1), win for the away team (or team 2, indicated with -1), or a draw (indicated with 0). In the case of knockouts, we limit the outcome to: win for the home team (or team 1), win for the away team (or team 2), as draws are not allowed.

To validate how accurate FIFA rankings are, we aim to use a baseline model that leverages FIFA rankings and some other simple predictors to predict the World Cup results. We plan to create a more advanced model, without relying on FIFA rankings but instead based on features collected and engineered by us that are proven to be relevant in the initial EDA. Ultimately, our analysis attempts to create a model that can predict the World Cup results as accurately as possible, while offering an insight into the features helpful in soccer analytics.

## Description of Data

**Rankings and outcomes**: As the baseline model relies on FIFA ranking, we obtained the rankings, points and the corresponding changes from [FIFA international men teams' rankings from August 1993 to June 2018](https://www.kaggle.com/tadhgfitzgerald/fifa-international-soccer-mens-ranking-1993now). FIFA outcomes between 1872 and 2017 dataset from Kaggle is used to construct the dependent variables for our training and validation sets. Similarly, we found [World Cup 2018 Stats](https://gitlab.com/djh_or/2018-world-cup-stats/blob/master/world_cup_2018_stats.csv) to construct our test set results. In addition, we scraped from [Wikipedia: World Cup Teams](https://en.wikipedia.org/wiki/2018_FIFA_World_Cup_squads) for countries involved in The World Cup at the given year (https://en.wikipedia.org/wiki/XXXX_FIFA_World_Cup_squads, where XXXX is the year).

**Player and team statistics**: Player data, which include ratings, positional data and wages, is provided from sofifa.com and also collected from other sources, such as [European matches](https://www.kaggle.com/hugomathien/soccer). However, as it was infeasible to reconcile the different naming conventions from different datasets for players' name along with a lot more players with the same name than we expected there to be (especially in the case of Brazil), it became an infesiable task to uniquely identify the players. This would require an extensive effort to manually curate the data, which would yield minimal benefit according to our EDA (see EDA section for details). Thus, we only aggregated the summary statistics of players data for each team to approximate the talent available in each team rather than inspecting the invidual statistics of each player.

**Country statistics**: Looking at the past World Cup champions, we noticed that the stronger teams tend to be the more economically wealthy ones in their own confederations. Therefore, we scraped from Wikipedia information about countries like [GDP PPP](https://en.wikipedia.org/wiki/List_of_countries_by_GDP_(PPP)) and [population](https://en.wikipedia.org/wiki/List_of_countries_by_population_(United_Nations) ).


**Detailed description of each collected feature** can be found in the following notebook [here](https://github.com/AC209aWC2018/World-Cup-2018/blob/master/doc/Feature%20Descriptions.ipynb)

#### Data Cleaning

We relied on `sqlite3` to query from European Soccer Database and web scrapers to scrape from Wikipedia. One of the key challenges involves matching different naming conventions for countries, players and teams used by each data source. In most cases, this was due to slight variation in spelling (i.e. St Lucia vs Saint Lucia). With all these different variations, we needed to map all the variations to just one name so that we could merge our datasets. We wrote the method `map_country()`, using [fuzzywuzzy](https://github.com/seatgeek/fuzzywuzzy), that essentially calculates the Levenshtein Distance for a country_name string if the country_name string does not exactly match with the FIFA names and map the string to the FIFA name with the highest similarity score (in percentage). We only do this mapping if the highest similarity score is above a 80%, when we are very confident that this is the right mapping. From manual inspection, this turned out extremely well and we believe that there should be no wrong mapping. However, in some cases, we still needed to do some manual mapping for ones in which the Levenshtein Distance is not confident about the mapping, such as in the case of the two Koreas, but the `map_country()` function has saved us from doing many manual mappings. 

In addition, there were teams that were not national teams at all (i.e. United Koreans in Japan, which is a team for the Korean population in Japan). However, as mentioned before, in the end, we only care about the national teams that are affiliated with FIFA. As such, we only keep the rows where both teams appeared on the FIFA rankings starting from 2006 (All affiliated teams are ranked, and we only look after 2006 because some countries that were on older rankings no longer exist or changed names such as Zaire, which became Congo DR). Our method `map_country()` in `map_countries.py` easily takes care of this since in general there is no similarity score that is above 80% (except for a few edge cases such as Northern Cyprus, which is a different team than Cyprus and is not allowed to compete in FIFA). These we manually took care of.

#### Feature Engineering

1. **Representation of home and away team**: Because in the case of the World Cup, home team does not actually mean anything (besides for Russia itself), we needed each feature in our models to be agnostic to which team is home and which team is away so that so that switching home team and away team does not result in different predictions despite everything else being the same. The simplest way to do so that also makes sense is to take the difference of home team features with away team features, as we can think of these as being difference in skills/abilities of the teams. 

2. **Missing value representation**: As some of the statistics, such as attack, defense and midfield ratings, we are interested in are only collected from FIFA 06 onwards, our training data starts from 2006. This has, for the most part, also solved the missing values issues present in the datasets. For the remaining missing values, we imputed them with 0 as our data is represented in differences between 2 teams, and 0 encodes the lack of information which gives neither advantage nor disadvantage to any team. 

3. **Momentum**: In sports, we can often notice that one team tends to keep winning or keep losing as they gain momentum due to teams' morale, conditions and other factors. This can be represented by taking into account exponential moving average of the number of wins in the past games. 

4. **Wage of players**: One of the most simplest and effective ways to judge the quality of a team is to calculate the total wage of the players in that team as clubs tend to pay higher wages for better players. For the players without wage report, we impute using the average wage in the country where the player belongs.

5. **Statistics of players**: We aggregate players' statitics together as part of a team's offensive, defensive abilities. One exception is the goalkeepers' ratings, as they are essential in preventing the opposing team from scoring and is the only category with a clearly defined role. Thus, we single out goalkeepers' ratings and include them as part of the features.

6. **GDP within the same confederation**: Rather than considering the overall GDP difference, GDP difference within the same confederation seems a better indicator of how well a team is doing against another. When not in the same confederation, this is encoded again with 0 to indicate the lack of information which gives neither advantage nor disadvantage to any team. 

## Exploratory Data Analysis

### European Soccer Database

Summary:

In this dataset, we focused primarily on seeing if individual player statistics were important predictors of the outcome of a match. We aggregated for each team different player statistics and differenced the aggregates. From looking at the plots the distributions of a difference in rating among the three outcome do not really seem that different. This might be an indication that using individual ratings and aggregating them might not be the way to go. 

![](img/european_league_differences.png)

## Exploring Other Features

![](img/spear_corr.png)
The spearman correlation plot gives us an idea of how well these features correlate with our dependent variable. In other words, the highly correlated features, such as overall difference in ratings should give us a fairly good idea of team 1/home team winning. 

![](img/distr_plot.png)
In fact, from the distribution plot we see that the features, such as overall difference in ratings and goalkeeper rating difference, that can better discriminate win, loss for team 1 are more highly correlated with the win/lose. Similarly, features, such as build up dribbling difference, which have low correlation barely show any difference between distribution of different outcomes.

![](img/goal_rating.png)
Inspecting one of the highly correlated features closely, we see that when the difference is positive (team 1 has an advantage), the likelihood of winning is much higher whereas when it's negative it's much more likely to lose. 

![](img/wage_diff.png)
However, there are also some features that don't have really high correlation and can still discriminate well the win/lose situations when the difference is large. For instance, for the team wage difference, we see when the difference is large, it is much more likely to win/lose, but there is barely any difference in the middle peaking at 0. This is happening because we imputed missing values with 0 to encode lack of information. In fact, in these cases we have a lot of distributions with a really high peak at 0, causing low correlation. Nonetheless, they can still be useful when we do possess the information. 

In addition, it seems that previous match results will be good predictors. This time it is a bit more obvious that home team winning tends to have higher values. It seems like we can probably calculate some kind of momentum and utilize it as a predictor. We will explore more of this concept later on when we try to make more advanced models.

![](img/previous_wins.png)

## Exploring PCA

Inspecting these distribution of features, we noticed them a large proportion of them are similarly shaped and distributed. Therefore, we considered PCA to perform dimensionality reduction, aggregating similar features.

To check how useful PCA is, we find the validation performance of cross validated models over different number of PCA components and compare it against the cross validated models on the data without PCA.

![](img/pca_res.png)

First of all, we notice that PCA helps models like LDA and QDA perhaps because of the independent assumption of components imposed by PCA. On the other hand, the performance for Logistic Regression, Random Forest, XGBoost seem to be slightly hurt by the reduced information from PCA. 

Nonetheless, we see that the best model is QDA with 9 components from PCA. This probably means that performing dimensionality reduction is probably quite helpful in summarizing the information from the data with similar distributions. We will keep this in mind when picking the final model. 

## Interpreting PCA

## Baseline Models

[Baseline Notebook can be found here](https://github.com/AC209aWC2018/World-Cup-2018/blob/master/models/baseline_model.ipynb)

We have two different simple baseline models. For the naive one, we just guess the majority outcome every time (home team wins). By just guessing the majority outcome every time, we could achieve a 42.1875% accuracy in predicting the World Cup results.

The other baseline model utilizes the FIFA rankings as well as some basic team features, including difference in attack ratings, difference in defense ratings, and difference in midfield ratings. Through classification methods such as logistic regression, LDA, QDA, Random Forest and XGBoost and cross validation, we picked LDA as the final baseline model as it gave the highest validation accuracy.

![](img/baseline_result.png)

With this model, we were able to achieve around 59.4% accuracy in predicting the 2018 World Cup results. We hope to at least match this accuracy in our more advanced model that does not utilize the FIFA rankings.

Note that because in the World Cup starting from the Round of 16 there can no longer be ties, we decided for these matches if the model predicts a tie (i.e. tie is the highest probability), we take the result with the second highest probability as our prediction. For this reason, the test accuracy is also higher because half of the matches can only have 2 outcomes instead of 3. 

